In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from torch import nn
from torchvision import models
import copy

device = torch.device("cuda")
# %env CUBLAS_WORKSPACE_CONFIG=:4096:8
os.environ["TORCH_HOME"] = "."
import sys

# torch.cuda.is_available()

from torchvision import models

############################################
# parameters to be set for encironment
# NOT NEED to modify if you have the setting

data_root_path = "/opt/local/torch/data/tcr-mat-form"  # path to store train data
util_path = "../../.."  # path to store the util package
############################################

# add local dir to sys path
sys.path.insert(0, util_path)  # the util package is supposed to be clone to this path

from util.model.surf.modified_cnn_model import (
    ModifiedPretrainedNet,
    SurfNet256,
)
from typing import Any, Dict, Union
from util.model.surf.dateset import SurfDatasetFromMat
from util.model.surf.pretrained_model import PretrainedModelDb
import torch.optim as optim


def define_surf_model(
    model_name: str,
    model_type: Union[int, str],
    suffix: str,
    pretrain: bool = True,
    set_type: str = "train",
    kwd: Dict[str, Any] = {
        "dropout": 0.2,
        "cnn_feature_ratio": 0.5,
        "data_root_path": "/hy-tmp/",
        "data_csv_filename": "DataNormilized.csv",
        "lr": 0.001,
        "num_params": 3,
        "num_output": 2,
    },
):
    """
    定义和配置一个用于表面模型的深度学习模型。

    参数:
    - model_name: 模型的名称。
    - model_type: 模型的类型，可以是整数或字符串。
    - suffix: 附加在模型名称和类型之后的后缀，用于区分不同的模型配置。
    - pretrain: 是否使用预训练的模型权重，默认为True。
    - kwd: 包含模型训练和配置所需的各种参数的字典。

    返回:
    一个包含模型配置和训练所需信息的字典，包括模型前缀、模型实例、数据集、优化器和损失函数。
    """

    # 初始化预训练模型数据库
    model_info_db = PretrainedModelDb()

    # 从数据库中获取指定模型和类型的信息
    train_model, model_weights, name_first_conv, name_fc = model_info_db.get_info(
        model_name, model_type
    )

    # 构造模型前缀，用于后续的日志记录或模型保存
    prefix = f"{model_name}{model_type}_{suffix}"

    # 创建一个修改过的预训练网络实例
    pnet = ModifiedPretrainedNet(
        pretrained_net=train_model,
        weights=model_weights if pretrain else None,
        name_first_conv=name_first_conv,
        name_fc=name_fc,
    )

    # 创建并配置最终的表面模型
    surf_model = SurfNet256(
        modified_net=pnet,
        num_params=kwd["num_params"],
        num_output=kwd["num_output"],
        dropout=kwd["dropout"],
        cnn_feature_ratio=kwd["cnn_feature_ratio"],
    )

    # 将模型移动到指定的设备上
    surf_model.to(device)

    # 创建并配置数据集
    dset = SurfDatasetFromMat(
        data_csv_filename=os.path.join(
            kwd["data_root_path"], set_type, kwd["data_csv_filename"]
        ),
        surf_data_dir=os.path.join(kwd["data_root_path"], set_type, "Surf"),
        param_start_idx=3,
        param_end_idx=6,
        num_targets=2,
    )

    # 创建优化器，用于模型参数的更新
    optimizer = optim.Adam(surf_model.parameters(), lr=kwd["lr"])

    # 定义损失函数，用于衡量模型预测值与真实值的差异
    loss_func = nn.MSELoss()

    # 返回包含模型配置和训练所需信息的字典
    return {
        "prefix": prefix,
        "train_model": surf_model,
        "dset": dset,
        "optimizer": optimizer,
        "loss_func": loss_func,
    }


kwd = {
    "dropout": 0.2,
    "cnn_feature_ratio": 0.5,
    "data_root_path": data_root_path,
    "data_csv_filename": "DataNormilized.csv",
    "lr": 0.001,
    "num_params": 3,
    "num_output": 2,
}
suffix = "input254_cv5_train10000"
model_info = define_surf_model(
    model_name="densenet",
    model_type="121",
    pretrain=False,
    suffix=suffix,
    set_type="test",
    kwd=kwd,
)
model_name = f"{model_info['prefix']}"
root_path = "../../../thesis/surfTopo/checkpoints"

best_checkpoint = torch.load(
    os.path.join(root_path, model_name, f"surf_{model_name}_best.ckpt")
)
# latest_checkpoint = torch.load(
#     os.path.join(root_path, model_name, f"surf_{model_name}_latest.ckpt")
# )
surf_model = model_info["train_model"]
surf_model.load_state_dict(best_checkpoint["model_state_dict"])
optimizer = model_info["optimizer"]
optimizer.load_state_dict(best_checkpoint["optimizer_state_dict"])
test_set = model_info["dset"]
loss_func = model_info["loss_func"]
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

In [2]:
from collections import OrderedDict
from typing import Tuple


class LayerVisualization:
    def __init__(
        self,
        model,
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
        verbose: bool = False,
    ):
        self.model = model
        self.device = device
        self.verbose = verbose

        self.model.eval()
        self.model.to(self.device)
        self.layer_info_dict = self.get_conv_layer_info(self.model)
        if self.verbose:
            self._show_layer_info()

        self._target_layer_conv_output = None
        self._all_layer_outputs = OrderedDict()

    def _show_layer_info(self):
        for name, info in self.layer_info_dict.items():
            print(
                f"{name}: The index is {info[0]} and the number of filters is {info[1]}"
            )

    @staticmethod
    def get_conv_layer_info(model: nn.Module) -> OrderedDict[str, Tuple[int, int]]:
        layer_info_dict = OrderedDict()
        i = 0
        for name, module in model.named_modules():
            if isinstance(module, nn.Conv2d):
                layer_info_dict[name] = (i, getattr(module, "out_channels"), module)
                i += 1
        return layer_info_dict

    def _hook_all_layers(self):
        def __create_hook(module_name):
            def __hook_func(module, grad_in, grad_out):

                self._all_layer_outputs[module_name] = grad_out[0]

            return __hook_func

        for name in self.layer_info_dict:
            if self.verbose:
                print(f"{name} is hooked!")
            self.layer_info_dict[name][2].register_forward_hook(__create_hook(name))

    def visual_all_layers(self, batch):
        self._hook_all_layers()
        batch = [item.to(self.device).requires_grad_() for item in batch]
        _ = self.model(*batch[0:-1])
        return self._all_layer_outputs

In [3]:
lv = LayerVisualization(model=surf_model, device="cuda", verbose=False)

In [4]:
for batch in test_loader:
    all_layers = lv.visual_all_layers(batch)
    break

### Use tensorboard to record the graph

In [ ]:
import torch.utils.tensorboard as tb
import torchvision.utils as vutils

writer = tb.SummaryWriter(f"./runs/CnnLayerVisualization")
for lname in all_layers:
    layer = all_layers[lname].unsqueeze(1).detach().cpu()
    lgrid = vutils.make_grid(
        layer, nrow=layer.shape[0] // 8, normalize=True, scale_each=True
    )
    writer.add_image(f"output of {lname}", lgrid)